In [1]:
# Import libraries
from dynamita.sumo import *

import numpy as np
import time
import copy as cp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
%matplotlib notebook

In [3]:
# Specify path where Sumo is found and Sumo license
sumo = Sumo(sumoPath="C:/Users/Sara/AppData/Local/Dynamita/Sumo19",
           licenseFile=r"C:/Users/Sara/Desktop/sewerWRRF/networklicense.sumolic")

License OK...


To load and run a dynamic input:
1. Save the dynamic input(s) in a .tsv file.
  - To make this easy, you can first do it from the Sumo GUI. This will create a .tsv file in the project's temporary directory (View > Directories > Project Directory) and "loadtsv ..." will be logged in the Core Window.
2. Copy the new initialization script and dynamic influent .tsv file into the working folder.
3. Make sure the command "loadtsv ... xxx.tsv" is in the initialization script. 
  - If it is not there, find it in the Core Window and paste it into the initialization script (the one in the working folder). It shouldn't matter where it is in the script.
  - Make sure the .tsv file path is updated to the working folder.
  - This step can also be done via calls in Python: `command = 'loadtsv '+ TSV +' ;'` and `sumo.core.csumo_command_send(sumo.handle, command.encode('utf8'))` where `TSV` is the .tsv file.

In [4]:
# Unload any models and load Sumo model of interest
sumo.unload_model()
#sumo.load_model('Sumo_Models/SUMO Model 03_recreated_additionChemP_primary_CSTRsmall.sumo')
sumo.load_model('Sumo_Models/MABR.sumo')

# Specify initialization script for Sumo model included above
#sumo.core.csumo_command_send(sumo.handle, b'execute script_Initialize_chemP_primary_CSTRsmall.scs;')

# Write commands for pulling and storing Sumo variables based on variable positions
# Note that these variables will be initiated as empty lists (e.g., t_set = []) and populated throughout the simulation
def datacomm_callback(sumo):
    # Simulation time step
    t_set.append(sumo.core.csumo_var_get_time_double(sumo.handle))
    
    # Influent flow
    q_infl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, q_infl_pos))
    # Influent total suspended solids (XTSS) concentration
    xtss_infl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, xtss_infl_pos))
    # Influent phosphate (SPO4) concentration
    spo4_infl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, spo4_infl_pos))
    # Influent total phosphorus (TP) concentration
    tp_infl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, tp_infl_pos))
    
    # After mixing (before primary clarifier) flow
    q_pi11_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, q_pi11_pos))
    # After mixing (before primary clarifier) total suspended solids (XTSS) concentration
    xtss_pi11_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, xtss_pi11_pos))
    # After mixing (before primary clarifier) phosphate (SPO4) concentration
    spo4_pi11_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, spo4_pi11_pos))
    # After mixing (before primary clarifier) total phosphorus (TP) concentration
    tp_pi11_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, tp_pi11_pos))
    
    # Effluent flow
    q_effl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, q_effl_pos))
    # Effluent total suspended solids (XTSS) concentration
    xtss_effl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, xtss_effl_pos))
    # Effluent phosphate (SPO4) concentration
    spo4_effl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, spo4_effl_pos))
    # Effluent total phosphorus (TP) concentration
    tp_effl_set.append(sumo.core.csumo_var_get_pvt_pos(sumo.handle, tp_effl_pos))
    
    return 0

sumo.register_datacomm_callback(datacomm_callback)

# Write function for printing Sumo commands (e.g., initiating and running model simulations)
def message_callback(sumo):
    for message in sumo.messages:
        print(message)
    sumo.messages = []
    return 0

sumo.register_message_callback(message_callback)

# Specify the length of the simulation and the frequency at which variables will be reported
# These are provided in milliseconds
sumo.set_stopTime(50*24*60*60*1000)
sumo.set_dataComm(50*60*1000)

No model is loaded
530021 Set: Sumo__StopTime to 0
530021 Set: Sumo__DataComm to 3600000
530021 Set: Sumo__PlantName to C:\Users\Sara\AppData\Local\Dynamita\Sumo19\.tmp\broxpubf.t0d\sumoproject.xml
530049 Core loop started.
530021 Set: Sumo__StopTime to 4320000000
530021 Set: Sumo__DataComm to 3000000


In [ ]:
# Store positions for variables of interest
q_infl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Influent__Q')
xtss_infl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Influent__XTSS')
spo4_infl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Influent__SPO4')
tp_infl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Influent__TP')

q_pi11_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Pipe11__Q')
xtss_pi11_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Pipe11__XTSS')
spo4_pi11_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Pipe11__SPO4')
tp_pi11_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Pipe11__TP')

q_effl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Effluent__Q')
xtss_effl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Effluent__XTSS')
spo4_effl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Effluent__SPO4')
tp_effl_pos = sumo.core.csumo_model_get_variable_info_pos(sumo.handle, b'Sumo__Plant__Effluent__TP')

### Change influent flow step value

In [ ]:
# Specify .tsv for reading (r) diurnal influent data and writing (w) changes to a new file
r_influentTSV = 'C:/Users/Sara/Desktop/sewerWRRF/SumoPythonPractice/Influent_Tables/Influent_Table_chemP_primary_steady.tsv'
w_influentTSV = 'C:/Users/Sara/Desktop/sewerWRRF/SumoPythonPractice/Influent_Tables/Influent_Table_chemP_primary_step.tsv'

# Read the original .tsv file as a pandas table
r_influentTSV_data = pd.read_table(r_influentTSV, sep='\t')

# Specify magnitude of step increases in influent flow
# Step increase will occur at (and continue after) simulation time step 20
# Steady-state (unchanged) is 2,788,860 m^3/d
stepMags_inflq = [-1000000, -500000, 0, 500000, 1000000]
# Specify magnitude of step increases in ferric chloride dose
# Step increase will occur at (and continue after) simulation time step 30
# Steady-state (unchanged) is 0.0 mg Fe/L
stepMags_femass = [1.5, 1.75, 2.0, 2.25, 2.5]
#stepMags_femass = [1.5, 2.0, 2.5]

femass_list = []; feconc_list = []
q_infl_list = []; q_pi11_list = []; q_effl_list = []
xtss_infl_list = []; xtss_pi11_list = []; xtss_effl_list = []
spo4_infl_list = []; spo4_pi11_list = []; spo4_effl_list = []
tp_infl_list = []; tp_pi11_list = []; tp_effl_list = []

In [ ]:
# Initiate figure for timeseries plots
fig, axes = plt.subplots(1,3, figsize=(10,4))

for j in stepMags_inflq:
    for k in stepMags_femass:
        
        print('Step magnitude influent flow: ' + str(j) + '\n')
        print('Step magnitude ferric chloride dose: ' + str(k) + '\n')
    
        # Make a change to the original .tsv file
        r_influentTSV_data = pd.read_table(r_influentTSV, sep='\t')
        
        inflq_steady = 2100000 #2788864.278
        femass_steady = 0.
    
        for i in range(0,50):
            r_influentTSV_data["Sumo__Plant__Influent__param__Q"].values[i] = inflq_steady
        for i in range(20,50):
            r_influentTSV_data["Sumo__Plant__Influent__param__Q"].values[i] += j
        
        for i in range(0,30):
            r_influentTSV_data["Sumo__Plant__Metal1__param__Femass"].values[i] = femass_steady
        for i in range(30,50):
            r_influentTSV_data["Sumo__Plant__Metal1__param__Femass"].values[i] = k*r_influentTSV_data["Sumo__Plant__Influent__param__Q"].values[i]
        
        
        # Write the changed .tsv file to a new file
        with open(w_influentTSV,'w') as write_tsv:
            write_tsv.write(r_influentTSV_data.to_csv(sep='\t', index=False))
        
        # Unload changed .tsv file to Sumo
        command = 'unloadtsv '+ w_influentTSV +' ;'
        sumo.core.csumo_command_send(sumo.handle, command.encode('utf8'))
        
        # Load changed .tsv file to Sumo
        command = 'loadtsv '+ w_influentTSV +' ;'
        sumo.core.csumo_command_send(sumo.handle, command.encode('utf8'))
        
        # Rerun Sumo simulation
        t_set = []
        q_infl_set = []; xtss_infl_set = []; spo4_infl_set = []; tp_infl_set = []
        q_pi11_set = []; xtss_pi11_set = []; spo4_pi11_set = []; tp_pi11_set = []
        q_effl_set = []; xtss_effl_set = []; spo4_effl_set = []; tp_effl_set = []
        
        sumo.run_model()
        
        while not sumo.simulation_finished:
            time.sleep(0.01)
        
        # Create dictionary of variables
        data_step = {}
        data_step['t'] = t_set
        data_step['q_infl'] = q_infl_set; data_step['q_pi11'] = q_pi11_set; data_step['q_effl'] = q_effl_set
        data_step['xtss_infl'] = xtss_infl_set; data_step['xtss_pi11'] = xtss_pi11_set; data_step['xtss_effl'] = xtss_effl_set
        data_step['spo4_infl'] = spo4_infl_set; data_step['spo4_pi11'] = spo4_pi11_set; data_step['spo4_effl'] = spo4_effl_set
        data_step['tp_infl'] = tp_infl_set; data_step['tp_pi11'] = tp_pi11_set; data_step['tp_effl'] = tp_effl_set
        
        # Create a pandas dataframe for this dictionary and index by time (t)
        df_step = pd.DataFrame.from_dict(data_step)
        df_step.set_index('t')
        
        # Store summaries
        femass_list.append(r_influentTSV_data["Sumo__Plant__Metal1__param__Femass"].values[-1])
        feconc_list.append(k)
        q_infl_list.append(data_step['q_infl'][-1]); q_pi11_list.append(data_step['q_pi11'][-1]); q_effl_list.append(data_step['q_effl'][-1])
        xtss_infl_list.append(data_step['xtss_infl'][-1]); xtss_pi11_list.append(data_step['xtss_pi11'][-1]); xtss_effl_list.append(data_step['xtss_effl'][-1])
        spo4_infl_list.append(data_step['spo4_infl'][-1]); spo4_pi11_list.append(data_step['spo4_pi11'][-1]); spo4_effl_list.append(data_step['spo4_effl'][-1])
        tp_infl_list.append(data_step['tp_infl'][-1]); tp_pi11_list.append(data_step['tp_pi11'][-1]); tp_effl_list.append(data_step['tp_effl'][-1])
        
        # Plot timeseries comparing influent and effluent of each step trial
        axes[0].plot(data_step['t'], data_step['q_infl'], ':', label="Influent, q_infl step: " + str(j) + ", femass: " + str(k))
        axes[0].plot(data_step['t'], data_step['q_pi11'], label="Pre 1clar, q_infl step: " + str(j) + ", femass: " + str(k))
        axes[0].plot(data_step['t'], data_step['q_effl'], label="Effluent, q_infl step: " + str(j) + ", femass: " + str(k))
        
        axes[1].plot(data_step['t'], data_step['spo4_infl'], ':', label="Influent, q_infl step: " + str(j) + ", femass: " + str(k))
        axes[1].plot(data_step['t'], data_step['spo4_pi11'], label="Pre 1clar, q_infl step: " + str(j) + ", femass: " + str(k))
        axes[1].plot(data_step['t'], data_step['spo4_effl'], label="Effluent, q_infl step: " + str(j) + ", femass: " + str(k))
        
        axes[2].plot(data_step['t'], data_step['tp_infl'], ':', label="Influent, q_infl step: " + str(j) + ", femass: " + str(k))
        axes[2].plot(data_step['t'], data_step['tp_pi11'], label="Pre 1clar, q_infl step: " + str(j) + ", femass: " + str(k))
        axes[2].plot(data_step['t'], data_step['tp_effl'], label="Effluent, q_infl step: " + str(j) + ", femass: " + str(k))

axes[0].set_xlabel('Time')
axes[0].set_ylabel('q')
axes[0].set_xlim(10,50)
#axes[0].legend()

axes[1].set_xlabel('Time')
axes[1].set_ylabel('spo4')
axes[1].set_xlim(10,50)
axes[1].set_ylim(0,2)
#axes[1].legend()

axes[2].set_xlabel('Time')
axes[2].set_ylabel('tp')
axes[2].set_xlim(10,50)
axes[2].set_ylim(0,3)
#axes[2].legend()

In [ ]:
data_step_summ = {}
data_step_summ['femass'] = femass_list; data_step_summ['feconc'] = feconc_list
data_step_summ['q_infl'] = q_infl_list; data_step_summ['q_pi11'] = q_pi11_list; data_step_summ['q_effl'] = q_effl_list
data_step_summ['xtss_infl'] = xtss_infl_list; data_step_summ['xtss_pi11'] = xtss_pi11_list; data_step_summ['xtss_effl'] = xtss_effl_list
data_step_summ['spo4_infl'] = spo4_infl_list; data_step_summ['spo4_pi11'] = spo4_pi11_list; data_step_summ['spo4_effl'] = spo4_effl_list
data_step_summ['tp_infl'] = tp_infl_list; data_step_summ['tp_pi11'] = tp_pi11_list; data_step_summ['tp_effl'] = tp_effl_list

df_step_summ = pd.DataFrame.from_dict(data_step_summ)

df_step_summ

In [ ]:
# Compare the step responses in spo4 effluent concentrations to the step increases in influent flow and ferric chloride dose
x = np.divide(df_step_summ['q_infl'],1000000)
y = df_step_summ['feconc']
z = df_step_summ['spo4_pi11']

fig, ax = plt.subplots(figsize=(5,4))
scat = ax.scatter(x, y, c=z, s=200, marker='o', cmap='inferno_r')
ax.set_xlabel('WRRF Infl Flow (Mm3/d)')
ax.set_ylabel('Ferric Chloride Dose (mg/L)')
ax.set_xticks((1.1,1.6,2.1,2.6,3.1))
cbar = fig.colorbar(scat)
cbar.ax.set_ylabel('Pre 1clar SPO4 (mg/L)')

In [ ]:
# Compare the step responses in spo4 effluent concentrations to the step increases in influent flow and ferric chloride dose
x = np.divide(df_step_summ['q_infl'],1000000)
y = df_step_summ['feconc']
z = df_step_summ['spo4_effl']

fig, ax = plt.subplots(figsize=(5,4))
scat = ax.scatter(x, y, c=z, s=200, marker='o', cmap='inferno_r')
ax.set_xlabel('WRRF Infl Flow (Mm3/d)')
ax.set_ylabel('Ferric Chloride Dose (mg/L)')
ax.set_xticks((1.1,1.6,2.1,2.6,3.1))
cbar = fig.colorbar(scat)
cbar.ax.set_ylabel('PE SPO4 (mg/L)')

In [ ]:
# Compare the step responses in spo4 effluent concentrations to the step increases in influent flow and ferric chloride dose
x = np.divide(df_step_summ['q_infl'],1000000)
y = df_step_summ['feconc']
z = df_step_summ['tp_effl']

fig, ax = plt.subplots(figsize=(5,4))
scat = ax.scatter(x, y, c=z, s=200, marker='o', cmap='inferno_r')
ax.set_xlabel('WRRF Infl Flow (Mm3/d)')
ax.set_ylabel('Ferric Chloride Dose (mg/L)')
ax.set_xticks((1.1,1.6,2.1,2.6,3.1))
cbar = fig.colorbar(scat)
cbar.ax.set_ylabel('PE TP (mg/L)')

In [ ]:
# Import sklearn packages
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
x = np.divide(df_step_summ['q_infl'],1000000)
y = df_step_summ['feconc']
z_tp_meas = df_step_summ['tp_effl']
z_sp_meas = df_step_summ['spo4_effl']

X = np.concatenate((np.array(x).reshape(-1,1),np.array(y).reshape(-1,1)),axis=1)

reg_tp = LinearRegression().fit(X, z_tp_meas)
print(reg_tp.score(X, z_tp_meas))
print(reg_tp.coef_)
print(reg_tp.intercept_)

reg_sp = LinearRegression().fit(X, z_sp_meas)
print(reg_sp.score(X, z_sp_meas))
print(reg_sp.coef_)
print(reg_sp.intercept_)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
x_new = np.linspace(1.1,3.1,num=51,endpoint=True)#[1.1,1.6,2.1,2.6,3.1]
y_new = np.linspace(2.5,1.5,num=51,endpoint=True)#[2.5,2.25,2,1.75,1.5]

x_new_mesh,y_new_mesh = np.meshgrid(x_new,y_new)

X = np.concatenate((np.array(x_new_mesh).reshape(-1,1),np.array(y_new_mesh).reshape(-1,1)),axis=1)

z_tp_pred = reg_tp.predict(X).reshape(x_new_mesh.shape)
z_sp_pred = reg_sp.predict(X).reshape(x_new_mesh.shape)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(z_tp_pred, cmap='inferno_r',extent=[x_new[0], x_new[-1], y_new[-1], y_new[0]], interpolation='bilinear')
ax.set_xlabel('WRRF Infl Flow (Mm3/d)')
ax.set_ylabel('Ferric Chloride Dose (mg/L)')
ax.set_xticks(x_new)
ax.set_yticks(y_new)
cbar = fig.colorbar(im, ax=ax)#, shrink=0.5)
cbar.ax.set_ylabel('PE TP (mg/L)')

x_line = [x_new[0],x_new[-1]]
z_line = [1.7,1.725,1.75,1.775,1.8]

coef_tp = reg_tp.coef_
inter_tp = reg_tp.intercept_

x_line,z_line = np.meshgrid(x_line,z_line)
y_line_tp = np.divide(np.subtract(np.subtract(z_line,np.multiply(x_line,coef_tp[0])),inter_tp),coef_tp[1])
print(y_line_tp)
for i in range(0,len(z_line)):
    ax.plot(x_line[i],y_line_tp[i],label=str(z_line[i,0]))
ax.legend()

ax.set_xlim(x_new[0],x_new[-1])
ax.set_ylim(y_new[-1],y_new[0])

ax.set_xticks((1.1,1.6,2.1,2.6,3.1))
ax.set_yticks((1.5,1.75,2.,2.25,2.5))

ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
plt.savefig("C:/Users/Sara/Desktop/sewerWRRF/data/results/stepheatmap_tp.svg")
plt.savefig("C:/Users/Sara/Desktop/sewerWRRF/data/results/stepheatmap_tp.png")

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(z_sp_pred, cmap='inferno_r',extent=[x_new[0], x_new[-1], y_new[-1], y_new[0]], interpolation='bilinear')
ax.set_xlabel('WRRF Infl Flow (Mm3/d)')
ax.set_ylabel('Ferric Chloride Dose (mg/L)')
ax.set_xticks(x_new)
ax.set_yticks(y_new)
cbar = fig.colorbar(im, ax=ax)#, shrink=0.5)
cbar.ax.set_ylabel('PE SPO4 (mg/L)')

x_line = [x_new[0],x_new[-1]]
z_line = [0.7,0.75,0.8,0.85]

coef_sp = reg_sp.coef_
inter_sp = reg_sp.intercept_

x_line,z_line = np.meshgrid(x_line,z_line)
y_line_sp = np.divide(np.subtract(np.subtract(z_line,np.multiply(x_line,coef_sp[0])),inter_sp),coef_sp[1])
print(y_line_sp)
for i in range(0,len(z_line)):
    ax.plot(x_line[i],y_line_sp[i],label=str(z_line[i,0]))
ax.legend()

ax.set_xlim(x_new[0],x_new[-1])
ax.set_ylim(y_new[-1],y_new[0])

ax.set_xticks((1.1,1.6,2.1,2.6,3.1))
ax.set_yticks((1.5,1.75,2.,2.25,2.5))

ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')

plt.savefig("C:/Users/Sara/Desktop/sewerWRRF/data/results/stepheatmap_spo4.svg")
plt.savefig("C:/Users/Sara/Desktop/sewerWRRF/data/results/stepheatmap_spo4.png")